In [3]:
from os import makedirs
from os.path import join
import logging
import numpy as np
import torch
import random
import sys

In [4]:
sys.path.append('../')
from torchsummary import summary
from torch.utils.data import DataLoader

In [5]:
from args import define_main_parser

from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

#from dataset.prsa import PRSADataset
from data.card import TransactionDataset
from models.modules import TabFormerBertLM
from scripts.utils import random_split_dataset
#from data.datacollator import TransDataCollatorForLanguageModeling
import data.datacollator as datacoll

In [6]:
from importlib import reload

In [7]:
reload(datacoll)

<module 'data.datacollator' from '/home/admin/murugesh/Clinical-Transformer/notebook/../data/datacollator.py'>

In [8]:
from argparse import Namespace
config = vars(Namespace(cached=False, checkpoint=0, data_extension='', data_fname='card_transaction.v3', data_root='./data/credit_card/', data_type='card', do_eval=False, do_train=True, field_ce=True, field_hs=64, flatten=False, jid=1, lm_type='bert', log_dir='sam/logs', mlm=True, mlm_prob=0.15, nrows=None, num_train_epochs=3, output_dir='sam', save_steps=500, seed=9, skip_user=False, stride=5, user_ids=None, vocab_file='vocab.nb'))
config['data_root'] = "../dataset/credit_card/"
config['output_dir'] = "sample"
config['log_dir'] = "sample/logs"
makedirs(config['output_dir'], exist_ok=True)
makedirs(config['log_dir'], exist_ok=True)

In [9]:
seed = config['seed']
random.seed(seed)  # python
np.random.seed(seed)  # numpy
torch.manual_seed(seed)  # torch
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)  # torch.cuda

In [10]:
dataset = TransactionDataset(root=config['data_root'],
                            fname=config['data_fname'],
                            fextension="",
                            vocab_dir=config['output_dir'],
                            nrows=None,
                            user_ids=None,
                            seq_len=20,
                            mlm=True,
                            cached=config['cached'],
                            stride=10,
                            flatten=config['flatten'],
                            return_labels=True,
                            skip_user=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.63it/s]


In [11]:
# #return_data = torch.tensor(dataset.data[0], dtype=torch.long)
# return_data = torch.tensor(dataset.data[0], dtype=torch.long).reshape(dataset.seq_len, -1)
# return_data2 = (return_data, torch.tensor(dataset.labels[0], dtype=torch.long))

In [13]:
vocab = dataset.vocab
custom_special_tokens = vocab.get_special_tokens()

totalN = len(dataset)
totalN = len(dataset)
trainN = int(0.6 * totalN)

valtestN = totalN - trainN
valN = int(valtestN * 0.5)
testN = valtestN - valN
lengths = [trainN, valN, testN]
print(f"# lengths: train [{trainN}]  valid [{valN}]  test [{testN}]")
print("# lengths: train [{:.2f}]  valid [{:.2f}]  test [{:.2f}]".format(trainN / totalN, valN / totalN,
                                                                               testN / totalN))

# lengths: train [17709]  valid [5903]  test [5903]
# lengths: train [0.60]  valid [0.20]  test [0.20]


In [14]:
train_dataset, eval_dataset, test_dataset = random_split_dataset(dataset, lengths)

In [19]:
collactor_cls = "TransDataCollatorForLanguageModeling"
data_collator = datacoll.TransDataCollatorForLanguageModeling(
        tokenizer=tab_net.tokenizer, mlm=True, mlm_probability=config['mlm_prob']
    )

In [20]:
train_dataloader = DataLoader(
            train_dataset,
            batch_size=200,
            collate_fn=data_collator)

In [17]:
# from tqdm import tqdm
# for inps in tqdm(train_dataloader):
#     print('out',inps['labels'])
#     pass

In [18]:
tab_net = TabFormerBertLM(custom_special_tokens,
                                  vocab=vocab,
                                  field_ce=config['field_ce'],
                                  flatten=config['flatten'],
                                  ncols=dataset.ncols,
                                  field_hidden_size=config['field_hs']
                                  )

In [21]:
training_args = TrainingArguments(
        output_dir=config['output_dir'],  # output directory
        num_train_epochs=config['num_train_epochs'],  # total number of training epochs
        logging_dir=config['log_dir'],  # directory for storing logs
        save_steps=config['save_steps'],
        do_train=config['do_train'],
        # do_eval=args.do_eval,
        # evaluation_strategy="epoch",
        prediction_loss_only=True,
        overwrite_output_dir=True,
        # eval_steps=10000
    )

In [297]:
#training_args

In [22]:
trainer = Trainer(
        model=tab_net.model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

In [299]:
#trainer.train()


In [300]:
train_dataset

In [23]:
model = tab_net.model
model = model.to('cuda:2')


In [24]:
optim_params = {'betas': (0.9, 0.999), 'eps': 1e-08, 'lr': 5e-05}
optim = torch.optim.AdamW(model.parameters(), **optim_params)

In [25]:
model = model.eval()

In [29]:
from tqdm import tqdm

In [30]:
for inps in tqdm(train_dataloader):
    print(inps.keys())
    print('inps', inps['input_ids'])
    print('masked_lm_labels',inps['masked_lm_labels'])
    break

  0%|                                                                                                      | 0/89 [00:00<?, ?it/s]/home/admin/murugesh/Clinical-Transformer/notebook/../data/datacollator.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labs = [torch.tensor(e[1], dtype=torch.long) for e in examples]
/home/admin/murugesh/Clinical-Transformer/notebook/../data/datacollator.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples = [torch.tensor(e[0], dtype=torch.long) for e in examples]
  0%|                                                                                                      | 0/89 [00:00<?, ?it/s]

dict_keys(['input_ids', 'masked_lm_labels', 'Ouput'])
inps tensor([[[    9,    14,    32,  ..., 10510, 10616,     1],
         [    9,    14,    27,  ..., 10513, 10616,     1],
         [    4,    14,    26,  ..., 10511, 10616,     1],
         ...,
         [    9,    14,    26,  ..., 10514, 10616,     1],
         [    4,    14,    23,  ..., 10515, 10616,     1],
         [    9,     4,    25,  ..., 10508, 10616,     1]],

        [[    7,    21,     4,  ...,     4, 10616,     1],
         [    7,    21,    32,  ..., 10510, 10616,     1],
         [    7,     4,    30,  ..., 10522, 10616,     1],
         ...,
         [    7,    21,    23,  ..., 10518, 10616,     1],
         [    7,    21,     4,  ..., 10507, 10616,     1],
         [    7,    21,    23,  ..., 10507, 10616,     1]],

        [[    9,    14,    29,  ..., 10539,     4,     1],
         [    9,    14,    30,  ..., 10507, 10616,     1],
         [    9,    14,    31,  ..., 10515, 10616,     1],
         ...,
         [

In [33]:
total_loss = 0
for inps in tqdm(train_dataloader):
    #print(inps.keys())
    #print(inps['input_ids'].shape)
    #print(inps['masked_lm_labels'].shape)
    #print(inps['masked_lm_labels'], )
    optim.zero_grad()
    labels = inps.pop("Ouput")
    model.train()
    inps['input_ids'] = inps['input_ids'].to('cuda:2')
    inps['masked_lm_labels'] = inps['masked_lm_labels'].to('cuda:2')
    outputs =model(**inps)
    #labels = inps.pop("labels")
    loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
    #loss.backward()
    #optim.step()
    total_loss += loss.item()
    #print(loss)
    # print(len(aa))
    # print("Length of out - ", len(out))
    # print('Ouput -', out[0].shape)
    # #print('Ouput -', out[1].shape)
    # print(aa[0])
    # print(aa[1].shape)
    # #aa.last_hidden_state
    # break


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:14<00:00,  6.17it/s]


In [34]:
print(total_loss/len(train_dataloader))

nan


In [79]:
#torch.save()

In [78]:
#trainer.train()

In [22]:
from transformers import BertModel

In [ ]:
model

In [30]:
model.save_pretrained('new_mode')

Configuration saved in new_mode/config.json
Model weights saved in new_mode/pytorch_model.bin


In [13]:
tab_net = TabFormerBertLM(custom_special_tokens,
                                  vocab=vocab,
                                  field_ce=config['field_ce'],
                                  flatten=config['flatten'],
                                  ncols=dataset.ncols,
                                  field_hidden_size=config['field_hs']
                                  )

In [34]:
#tab_net.model.tb_model.save_pretrained('new_mode')

In [50]:
loaded_val = torch.load('new_mode/pytorch_model.bin')

In [51]:
model.load_state_dict(loaded_val)

<All keys matched successfully>

In [30]:
#tab_net.model.tb_model = tab_net.model.tb_model.from_pretrained('new_mode')

In [31]:
#summary(tab_net.model.tb_model)

In [32]:
#tab_net.model.tb_model.bert.encoder

In [37]:
from models.lstm_classifier import LSTM

In [28]:
bert_fe_model = tab_net.model

In [38]:
classifier_model = LSTM(emb_inp_size=1062)

In [39]:
import torch.nn as nn


In [40]:
tab_net = TabFormerBertLM(custom_special_tokens,
                                  vocab=vocab,
                                  field_ce=config['field_ce'],
                                  flatten=config['flatten'],
                                  ncols=dataset.ncols,
                                  field_hidden_size=config['field_hs']
                                  )

In [302]:
train_dataloader_cls = DataLoader(
            train_dataset,
            batch_size=200)

In [75]:
class Classifier(nn.Module):
    def __init__(self, custom_special_tokens,
                 vocab,
                 field_ce,
                 flatten,
                 ncols,
                 field_hidden_size,
                 bert_feature_size,
                 base_model
                 ):
        super(Classifier, self).__init__()
        '''
        self.tab_net = TabFormerBertLM(custom_special_tokens,
                                        vocab=vocab,
                                        field_ce=field_ce,
                                        flatten=flatten,
                                        ncols=ncols,
                                        field_hidden_size=field_hidden_size)
        '''
        loaded_val = torch.load('new_mode/pytorch_model.bin')
        base_model.load_state_dict(loaded_val)
        #print(base_model)
        for p in base_model.parameters():
            p.requires_grad = True
        self.field_transformer = base_model.tab_embeddings
        self.bert = base_model.tb_model
        


        self.classifier = LSTM(emb_inp_size=bert_feature_size)

    
    def forward(self, input_ids ,input_args):
        field_embeddings = self.field_transformer(input_ids)
        #input_args['input_ids'] = input_ids
        bert_features = self.bert(inputs_embeds=field_embeddings, **input_args)
        
        bert_features = bert_features[1]
        bert_features = bert_features.reshape((10, 20, 11, 1062))
        bert_features = bert_features.reshape((200, 11, 1062))
        cls_out = self.classifier(bert_features, T.as_tensor(([11])))
        return cls_out
    

In [76]:
classif = Classifier(custom_special_tokens,
                 vocab=vocab,
                    field_ce=config['field_ce'],
                    flatten=config['flatten'],
                    ncols=dataset.ncols,
                    field_hidden_size=config['field_hs'],
                 bert_feature_size=1062, base_model=model)

In [70]:
#model.tb_model

In [99]:
import torch as T
T.as_tensor(([232]))

tensor([232])

In [309]:
for inps in train_dataloade:
    input_ids = inps.pop('input_ids')
    #print(out.shape)
    #print(input_ids.shape)
    #print(inps['masked_lm_labels'].shape)
    #print(inps['masked_lm_labels'], )
    print(inps[0].shape)
    print(inps[1].shape)
    class_out =classif(input_ids, inps)
    
    print(f"Class out - {class_out.shape}")
    print(class_out)
    break

torch.Size([200, 20])
torch.Size([20, 11])
torch.Size([20, 11])


NameError: name 'classif' is not defined

1. append classifier with bert
2. Freeze bert model after first train
3. Use bert model and train the classifier